In [1]:
from tokenizer import Tokenizer
import torch
import torch.nn as nn

In [38]:
obj = Tokenizer()
obj.character_tokenizer()
data=obj.encoder(s=obj.data)

In [39]:
n=int(0.9*len(data))
traindata = data[:n]
intrimdata = data[n:]

In [40]:
block_size = 128  # context length
batch_size = 64   # number of sequences in a batch

data = torch.tensor(traindata) 
ix = torch.randint(len(data) - block_size, (batch_size,))
x = torch.stack([data[i:i+block_size] for i in ix])
y = torch.stack([data[i+1:i+block_size+1] for i in ix])


In [41]:
embeding=nn.Embedding(embedding_dim=8,num_embeddings=obj.vocab_size)

In [42]:
val=obj.encoder(s='hello')
val

[64, 61, 68, 68, 71]

In [43]:
embeding(torch.tensor(val))

tensor([[-0.2348, -0.9463, -0.3062,  0.5199,  2.5804, -1.1007,  1.0478,  1.3342],
        [-1.1583, -1.4956, -1.5983,  0.4800, -1.6429, -2.1345, -0.4105,  0.4905],
        [ 0.4344, -1.9602, -0.1306,  0.1736, -0.2423,  1.2940, -0.2329,  2.1978],
        [ 0.4344, -1.9602, -0.1306,  0.1736, -0.2423,  1.2940, -0.2329,  2.1978],
        [ 0.0867, -0.1407, -0.6312,  1.3012,  0.8746,  0.7028,  1.8270, -0.8602]],
       grad_fn=<EmbeddingBackward0>)

In [44]:
int(x[0][0])

69

In [46]:
print( obj.decoder(x[0]))
    

KeyError: tensor(69)